## 최대 입력토큰까지 밀어넣는 방법으로 최대한 적게 사용할것
## 메일별 토큰사이즈를 예측하는 코드 필요
## GPT에게 시킨 말을 최적화 하는 기능이 필요
## GPT로 검증이 되면 라마로 이동

In [59]:
from openai import OpenAI
import json

In [60]:
# 한번 업데이트되어부렸으
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="sk-_OB5eQftpw-yvfeqrU_w5wFMaInfDVHu98YDSKW6oHT3BlbkFJugbrClXorfnjs3QAL2SGP2kTRUCVgGrmdFvBkxmlcA",
)

In [61]:
# 함수 정의 - 날씨와 주식 정보를 반환
def get_weather(location):
    return f"{location}의 날씨는 맑습니다."

def get_stock_info(company_name):
    return f"{company_name}의 주식은 상승 중입니다."

In [63]:
# JSON Schema와 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Provides the current weather information for a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The location to get weather information for."}
                },
                "required": ["location"],
                "additionalProperties": False
            },
            "strict": True  # Structured Outputs 사용
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_stock_info",
            "description": "Provides the current stock status for a given company.",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {"type": "string", "description": "The company name to get stock information for."}
                },
                "required": ["company_name"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

In [64]:
# 사용자 요청 메시지 정의
messages = [
    {"role": "system", "content": "You are an assistant that provides weather and stock information based on user input."},
    {"role": "user", "content": "서울의 날씨를 알려줘"},
    {"role": "user", "content": "삼성전자의 주식 현황을 알려줘"}
]

In [66]:
# ChatCompletion 호출 - 도구 자동 선택
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto"  # 'auto'로 설정하여 GPT가 도구를 자동으로 선택하게 함
)

In [69]:
# 함수 호출이 발생한 경우 결과 처리
for choice in response.choices:
    tool_calls = choice.message.tool_calls
    
    for tool_call in tool_calls:
        # 함수 호출의 이름과 인수 가져오기
        function_name = tool_call.function.name
        data = tool_call.function.arguments 
#         data = data.encode().decode('unicode_escape')  # 과거버전에서만 필요.. 대단하네 바로나오네
        arguments = json.loads(data)
        
        # 함수 실행 및 결과 출력
        if function_name == "get_weather":
            result = get_weather(arguments['location'])
        elif function_name == "get_stock_info":
            result = get_stock_info(arguments['company_name'])
        
        print(f"Result from {function_name}: {result}")

Result from get_weather: 서울의 날씨는 맑습니다.
Result from get_stock_info: 삼성전자의 주식은 상승 중입니다.
